# Imports

In [3]:
import requests
from bs4 import BeautifulSoup

# Constant

In [4]:
PAGE_URL = 'https://www.zocdoc.com/search?address=New%20Jersey'

# Code

In [10]:
# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

cookies = {
    'firstTimeVisitor': '1e8d84b5-507c-4995-9b3d-ecceb5f5b7a0',
    'originalReferrer': 'NONE',
    'mostRecentReferrer': 'NONE',
    'ABG': '51d7cb0f-81e6-4de7-a312-f77bf654f713',
    'abfp': '1',
    'ASP.NET_SessionId': 'kx51dmlvaxdmmvmfzmphbcdj',
    'SearchLocation': '{}',
    'address_cookie': 'New%20Jersey',
    'hCarrierId': '-1',
    'hPlanId': '-1',
    'isDefaultHPlan': 'false',
    'previouslySearchedProcedureIdForTriage': '75',
    'AWSALB': 'HObpfc8Sa7mQJ8zG9jT0gQSuQt+dYCqS9Y19NgsQLGyle51juuYSb6Sr6IPwpVZ3X/zSI4fxdslkAgSTvroXPfDjKENCLZDoKs75FIL56y9dkWZ3r+dyo3LFaqOY',
    'AWSALBCORS': 'HObpfc8Sa7mQJ8zG9jT0gQSuQt+dYCqS9Y19NgsQLGyle51juuYSb6Sr6IPwpVZ3X/zSI4fxdslkAgSTvroXPfDjKENCLZDoKs75FIL56y9dkWZ3r+dyo3LFaqOY',
    'isNewPatient': '2024-05-21T16:56:40.858Z',
    'bsid': '34a56faf23ff41118512ab6899df08df_2405201729',
    'datadome': 'IjP62X1LUaqRPrUbS08MjZcqf3muuPlrruMbfuRw7POQRDrBvze~XpYpSuFgwMNSado4sxWT_6cOJXfu0347T0bHJbByvbCl2u4Jgpl8gBBMnkO0sR6iX70TFgf5pwQM',
    'lux_uid': '171622614456588604',
    'referrer_34a56faf23ff41118512ab6899df08df': 'www.zocdoc.com%2F',
}


# Send a GET request to the website with headers
response = requests.get(PAGE_URL, headers=headers, cookies=cookies)

# # Check if the request was successful
# if response.status_code == 200:
#     soup = BeautifulSoup(response.content, 'html.parser')
    
# #     # Find the <body> tag
# #     body_tag = soup.find('body')

# #     # Print the contents of the <body> tag
# #     if body_tag:
# #         print(body_tag.prettify())
# #     else:
# #         print("No <body> tag found in the HTML.")

#     # Find the <div id="root"> tag
#     root_div = soup.find('div', id='root')

#     # Print the content of the <div id="root"> tag
#     if root_div:
#         print(root_div.prettify())
#     else:
#         print('No <div id="root"> tag found')
# else:
#     print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <article> tags with the attribute data-location-id
articles = soup.find_all('article', attrs={'data-location-id': True})

# Print the content of each <article> tag found
for article in articles:
    print(article.prettify())

<article class="sc-s7kkde-5 cMeSLG" data-location-id="pr_FVRtdpY77EOYzpGLNGHQfB|lo_KAXDdf-qJ0aEnduKEecSHR" data-test="search-result-item-ad" data-uem-id="search-result-item-298558_136095">
 <div class="sc-s7kkde-2 incfq">
  <div class="sc-s7kkde-1 dDJUF">
   <div class="sc-bHnlcS grgspR" data-is-sponsored-result="true" data-test="doctor-card" data-uem-id="sponsored-result-item">
    <div data-test="lazyload-tracking-pixel">
    </div>
    <div class="sc-cKXybt bgEaxj">
     <div class="sc-lgjHQU jTBQzm">
      <div class="sc-iQbOkh imhSLC" data-test="polaris-doctor-photo">
       <div class="sc-1w55mud-0 dZFdGN">
        <div data-test="lazyload-js-avatar" style="height:96px;width:96px">
        </div>
        <noscript data-test="default-doctor-photo-noscript">
         <div class="sc-7juqbx-0 feWUBg" data-test="doctor-photo-circle-mask" size="96">
          <img alt="Dr. Debbie Goodman" data-test="default-doctor-photo" height="96" itemprop="image" loading="eager" src="//d2uur722ua7fv

In [6]:
response.status_code

200